In [51]:
import requests
from bs4 import BeautifulSoup
import time
import os
import json
import re

In [53]:
def salvarPagina(pagina):
    response = requests.get(pagina)
    soup = BeautifulSoup(response.content, "html.parser")
    titulo = soup.select(".mw-page-title-main")
    if not titulo:
        nome_arquivo = pagina.split("/wiki/")[-1].replace("_", " ")  
    else:
        nome_arquivo = sanitizar(titulo[0].text)


    os.makedirs("wikipediapaginas", exist_ok=True)
    
    with open(f"wikipediapaginas/{nome_arquivo}.html", "w", encoding="utf-8") as g:
        conteudo = response.content.decode("utf-8")
        g.write(conteudo)

    return soup;

In [55]:
def sanitizar(nomearquivo):
    invalido = r'[<>:"/\\|?*\x00-\x1F]'
    nomearquivo = re.sub(invalido,"", nomearquivo)
    nomearquivo = nomearquivo.strip('. ')
    return nomearquivo

In [57]:
def adicionarLinks (linksAvisitar, novosLinks, linksVisitados):
    if len(linksVisitados) >= 5000:
        return True
    else:
        for a in novosLinks:
            if a not in linksAvisitar and a not in linksVisitados:
                linksAvisitar.append(a)

    return False

In [59]:
def visitarPagina(pagina, linksVisitados, linksAvisitar):
    response = requests.get(pagina)
    soup = salvarPagina(pagina)   #salva pagina
    
    todos_links = soup.find_all("a")
    novosLinks = []
    for link in todos_links: #extrai e filtra os links
        if "href" in link.attrs.keys() and link["href"].startswith("/wiki/"):
            if link["href"].find(":") != -1:
                continue
            
            novosLinks.append(f"https://pt.wikipedia.org{link['href']}")
            #print(links[-1])
    
    #print(f"Encontrados {len(links)} links nesta página -- {len(set(links))}")
    
    fimDoLoop = adicionarLinks(linksAvisitar, novosLinks, linksVisitados) #guarda na lista os novos links
    
    linksVisitados.append(pagina) #poem a pagina ja vistiada na lista de paginas ja visitadas
    return fimDoLoop

In [61]:
fimDoLoop = False
pagina = "https://pt.wikipedia.org/wiki/Wikip%C3%A9dia:P%C3%A1gina_principal"
linksAvisitar = []
linksVisitados = []
linksAvisitar.append(pagina)
i=0
while(fimDoLoop == False):
    pagina = linksAvisitar[i]
    fimDoLoop = visitarPagina(pagina, linksVisitados, linksAvisitar)
    time.sleep(1)
    i+=1